In [25]:
!git clone https://github.com/Dinith132/steel-industry-energy-consumption.git

Cloning into 'steel-industry-energy-consumption'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 77 (delta 7), reused 22 (delta 6), pack-reused 54 (from 1)
Receiving objects: 100% (77/77), 61.65 MiB | 31.42 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [26]:
cd steel-industry-energy-consumption/with_down/Training/

/content/steel-industry-energy-consumption/with_down/Training/steel-industry-energy-consumption/with_down/Training


In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [28]:
df=pd.read_csv("../data_log_all.csv")

In [29]:
df.head()

,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Load_Type
0,0.068186,0.352315,0.0,0.0,0.638592,1.0,0.000000,0.0,0.0
1,0.070905,0.372114,0.0,0.0,0.585156,1.0,0.043478,0.0,0.0
2,0.072554,0.382602,0.0,0.0,0.557186,1.0,0.086957,0.0,0.0
3,0.068034,0.377244,0.0,0.0,0.562310,1.0,0.130435,0.0,0.0
4,0.077733,0.393285,0.0,0.0,0.537829,1.0,0.173913,0.0,0.0


In [30]:
# Sliding window transformation
def create_sliding_windows(data, window_size=1):
    X, y = [], []
    for i in range(len(data)-window_size):
        X.append(data[i:i+window_size, 1:])
        y.append(data[i+window_size, 0])
    return np.array(X), np.array(y)

In [31]:
data={}

In [32]:
df_np = df.to_numpy()

In [33]:
window_sizes=[1,4,8,12,16,24,36,48,74,168,336,672]

In [34]:
for window in window_sizes:
  X, y = create_sliding_windows(df_np, window)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, shuffle=False)
  data[f"win{window}"] = {
      "X_train": X_train,
      "X_test": X_test,
      "y_train": y_train,
      "y_test": y_test
  }

In [35]:
data['win1']['X_train'].shape, data['win1']['y_train'].shape, data['win1']['X_test'].shape, data['win1']['y_test'].shape

((7182, 1, 8), (7182,), (1577, 1, 8), (1577,))

trainig

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout

In [37]:
def build_lstm_model(model_type, input_shape):
    model = Sequential()

    if model_type == 'single':
        model.add(LSTM(64, input_shape=input_shape))
    elif model_type == 'double':
        model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
        model.add(LSTM(64))
    elif model_type == 'bidirectional':
        model.add(Bidirectional(LSTM(64), input_shape=input_shape))

    model.add(Dropout(0.1))
    model.add(Dense(1))
    return model

In [38]:
import tensorflow.keras.backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

In [39]:
# prompt: mount the disk

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
from tensorflow.keras.models import load_model

# Save model to disk
def save_model(model, window_size):
    path = f"{model_dir}/win{window_size}.h5"
    model.save(path)

# Load model from disk
def load_saved_model(window_size):
    path = f"{model_dir}/win{window_size}.h5"
    if os.path.exists(path):
        return load_model(path, custom_objects={'rmse': rmse})
    else:
        return None

Single layer


In [41]:
import os

model_dir = '/content/drive/MyDrive/reserach/with_down/log_all/single'
os.makedirs(model_dir, exist_ok=True)

In [42]:
single_models = {}

for window_size in window_sizes:
    print(f"==> Window: {window_size}")

    X_train = data[f"win{window_size}"]["X_train"]
    X_test = data[f"win{window_size}"]["X_test"]
    y_train = data[f"win{window_size}"]["y_train"]
    y_test = data[f"win{window_size}"]["y_test"]

    print("-->", X_train.shape, y_train.shape, X_test.shape)

    # Try to load the model
    model = load_saved_model(window_size)

    if model is not None:
        print(f"✅ Loaded existing model for window {window_size}")
        history = None  # Since we didn't train now
    else:
        print(f"🚀 Training new model for window {window_size}")
        model = build_lstm_model('single', (window_size, X_train.shape[2]))
        model.compile(optimizer='adam', loss=rmse, metrics=['mae'])
        history = model.fit(
            X_train, y_train,
            epochs=1,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        save_model(model, window_size)
        print(f"💾 Model saved for window {window_size}")

    single_models[f"win{window_size}"] = {
        "model": model,
        "history": history
    }


==> Window: 1
--> (7182, 1, 8) (7182,) (1577, 1, 8)
🚀 Training new model for window 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.3342 - mae: 0.2731 - val_loss: 0.1799 - val_mae: 0.1439


💾 Model saved for window 1
==> Window: 4
--> (7179, 4, 8) (7179,) (1577, 4, 8)
🚀 Training new model for window 4
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.2658 - mae: 0.2198 - val_loss: 0.1855 - val_mae: 0.1538


💾 Model saved for window 4
==> Window: 8
--> (7176, 8, 8) (7176,) (1576, 8, 8)
🚀 Training new model for window 8
180/180 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.2560 - mae: 0.2144 - val_loss: 0.1554 - val_mae: 0.1191


...

💾 Model saved for window 48
==> Window: 74
--> (7122, 74, 8) (7122,) (1564, 74, 8)
🚀 Training new model for window 74
179/179 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 0.3008 - mae: 0.2486 - val_loss: 0.1631 - val_mae: 0.1247


💾 Model saved for window 74
==> Window: 168
--> (7045, 168, 8) (7045,) (1547, 168, 8)
🚀 Training new model for window 168
177/177 ━━━━━━━━━━━━━━━━━━━━ 15s 69ms/step - loss: 0.2988 - mae: 0.2522 - val_loss: 0.1611 - val_mae: 0.1227


💾 Model saved for window 168
==> Window: 336
--> (6907, 336, 8) (6907,) (1517, 336, 8)
🚀 Training new model for window 336
173/173 ━━━━━━━━━━━━━━━━━━━━ 30s 161ms/step - loss: 0.2775 - mae: 0.2350 - val_loss: 0.1694 - val_mae: 0.1307


💾 Model saved for window 336
==> Window: 672
--> (6632, 672, 8) (6632,) (1456, 672, 8)
🚀 Training new model for window 672
166/166 ━━━━━━━━━━━━━━━━━━━━ 48s 276ms/step - loss: 0.2342 - mae: 0.1960 - val_loss: 0.1694 - val_mae: 0.1266


💾 Model saved for window 672


double

In [43]:
model_dir = '/content/drive/MyDrive/reserach/with_down/log_all/double'
os.makedirs(model_dir, exist_ok=True)

In [44]:
double_models = {}

for window_size in window_sizes:
    print(f"==> Window: {window_size}")

    X_train = data[f"win{window_size}"]["X_train"]
    X_test = data[f"win{window_size}"]["X_test"]
    y_train = data[f"win{window_size}"]["y_train"]
    y_test = data[f"win{window_size}"]["y_test"]

    print("-->", X_train.shape, y_train.shape, X_test.shape)

    # Try to load the model
    model = load_saved_model(window_size)

    if model is not None:
        print(f"✅ Loaded existing model for window {window_size}")
        history = None  # Since we didn't train now
    else:
        print(f"🚀 Training new model for window {window_size}")
        model = build_lstm_model('double', (window_size, X_train.shape[2]))
        model.compile(optimizer='adam', loss=rmse, metrics=['mae'])
        history = model.fit(
            X_train, y_train,
            epochs=1,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        save_model(model, window_size)
        print(f"💾 Model saved for window {window_size}")

    single_models[f"win{window_size}"] = {
        "model": model,
        "history": history
    }


==> Window: 1
--> (7182, 1, 8) (7182,) (1577, 1, 8)
🚀 Training new model for window 1
180/180 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.3274 - mae: 0.2646 - val_loss: 0.1782 - val_mae: 0.1388


💾 Model saved for window 1
==> Window: 4
--> (7179, 4, 8) (7179,) (1577, 4, 8)
🚀 Training new model for window 4
180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.2808 - mae: 0.2346 - val_loss: 0.1833 - val_mae: 0.1452


💾 Model saved for window 4
==> Window: 8
--> (7176, 8, 8) (7176,) (1576, 8, 8)
🚀 Training new model for window 8
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 0.2583 - mae: 0.2140 - val_loss: 0.1741 - val_mae: 0.1267


💾 Model saved for window 8
==> Window: 12
--> (7173, 12, 8) (7173,) (1575, 12, 8)
🚀 Training new model for window 12
174/180 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.2662 - mae: 0.2219

KeyboardInterrupt: 

bi dir

In [ ]:
model_dir = '/content/drive/MyDrive/reserach/with_down/log_all/bidir'
os.makedirs(model_dir, exist_ok=True)

In [ ]:
bidir_models = {}

for window_size in window_sizes:
    print(f"==> Window: {window_size}")

    X_train = data[f"win{window_size}"]["X_train"]
    X_test = data[f"win{window_size}"]["X_test"]
    y_train = data[f"win{window_size}"]["y_train"]
    y_test = data[f"win{window_size}"]["y_test"]

    print("-->", X_train.shape, y_train.shape, X_test.shape)

    # Try to load the model
    model = load_saved_model(window_size)

    if model is not None:
        print(f"✅ Loaded existing model for window {window_size}")
        history = None  # Since we didn't train now
    else:
        print(f"🚀 Training new model for window {window_size}")
        model = build_lstm_model('bidirectional', (window_size, X_train.shape[2]))
        model.compile(optimizer='adam', loss=rmse, metrics=['mae'])
        history = model.fit(
            X_train, y_train,
            epochs=1,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        save_model(model, window_size)
        print(f"💾 Model saved for window {window_size}")

    single_models[f"win{window_size}"] = {
        "model": model,
        "history": history
    }
